# Coursera IBM Data Science Certificate
## Capstone Project week 4+5

#### Introduction/Business Problem
My client is a family who's lived in New York for the last 10 years. Husband and wife have a good reputation on the local job market and were able to transition fully into home office working arrangements. They have gotten quite tired of the many people in New York, however they do appreciate its cultural offerings, bars, parks and general characteristics. They decided to have children soon and need to move to a smaller city. Thus, they asked me to analyze the top 50 cities in the US by population and find one that has the same flair and characteristics as NY, however with fewer people. 